<a href="https://colab.research.google.com/github/vibhuverma17/MLBASEDSAMPLING/blob/main/TUANDROMD_Sampling_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install umap-learn hdbscan gower kmodes XGBoost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.1 MB/s eta 0:00:00


In [2]:
# ===============================
# Standard Libraries
# ===============================
import time
import numpy as np
import pandas as pd
import warnings
import ast

# ===============================
# Visualization Libraries
# ===============================
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

# ===============================
# Scikit-learn Components
# ===============================
# Data Splitting and Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

# Models
from sklearn.ensemble import (
    RandomForestClassifier, RandomForestRegressor,
    GradientBoostingClassifier, GradientBoostingRegressor,
    IsolationForest
)
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.cluster import KMeans

# Metrics
from sklearn.metrics import (
    f1_score, roc_auc_score, accuracy_score, recall_score, precision_score,
    mean_absolute_error, mean_squared_error, r2_score,
    mean_absolute_percentage_error, roc_curve
)

# Pairwise distances
from sklearn.metrics.pairwise import pairwise_distances

# ===============================
# External Libraries
# ===============================
import xgboost as xgb  # XGBoost library
from scipy.stats import ks_2samp, entropy  # Statistical tests
from scipy.spatial.distance import cdist
from kmodes.kprototypes import KPrototypes  # Clustering
import umap  # Dimensionality reduction
import hdbscan  # Density-based clustering
import gower  # Gower similarity for mixed data types

# ===============================
# Configure Warnings
# ===============================
warnings.filterwarnings("ignore")

#### DIM REDUCTION AND GRID SEARCH

In [3]:
class StratifiedSamplingUMAP:
    def __init__(self, n_neighbors=15, min_dist=0.1, n_components=2, n_grids=10, sample_percentage=0.1):
        self.n_neighbors = n_neighbors
        self.min_dist = min_dist
        self.n_components = n_components
        self.n_grids = n_grids
        self.sample_percentage = sample_percentage
        self.sample_indices = None  # To store sampled indices

    def check_categorical(self, data):
        return data.select_dtypes(include=['object']).shape[1] > 0

    def fit_transform(self, data):
        if self.check_categorical(data):
            print("Categorical variables detected. Using 'dice' metric for UMAP.")
            categorical_data = data.select_dtypes(include=['object'])
            encoder = OneHotEncoder(sparse_output=False)
            categorical_encoded = encoder.fit_transform(categorical_data)
            continuous_data = data.select_dtypes(exclude=['object'])
            combined_data = np.hstack([categorical_encoded, continuous_data])
            metric = 'dice'
        else:
            print("No categorical variables detected. Using 'euclidean' metric for UMAP.")
            combined_data = data
            metric = 'euclidean'

        umap_model = umap.UMAP(n_neighbors=self.n_neighbors, min_dist=self.min_dist,
                               n_components=self.n_components, metric=metric)
        return umap_model.fit_transform(combined_data)

    def stratified_sampling(self, embedding):
        min_x, min_y = np.min(embedding[:, :2], axis=0)
        max_x, max_y = np.max(embedding[:, :2], axis=0)

        x_bins = np.linspace(min_x, max_x, self.n_grids)
        y_bins = np.linspace(min_y, max_y, self.n_grids)

        if self.n_components == 3:
            min_z = np.min(embedding[:, 2])
            max_z = np.max(embedding[:, 2])
            z_bins = np.linspace(min_z, max_z, self.n_grids)
            grid_counts = np.zeros((self.n_grids, self.n_grids, self.n_grids))
        else:
            grid_counts = np.zeros((self.n_grids, self.n_grids))

        for point in embedding:
            x_idx = np.digitize(point[0], x_bins) - 1
            y_idx = np.digitize(point[1], y_bins) - 1

            if self.n_components == 3:
                z_idx = np.digitize(point[2], z_bins) - 1
                grid_counts[x_idx, y_idx, z_idx] += 1
            else:
                grid_counts[x_idx, y_idx] += 1

        grid_probs = grid_counts / np.sum(grid_counts)
        grid_probs_flat = grid_probs.flatten()

        # Sample indices based on the probability distribution
        sampled_indices = np.random.choice(len(embedding), size=int(len(embedding) * self.sample_percentage), replace=False)

        # Store the sampled indices for later retrieval
        self.sample_indices = sampled_indices

        # Extract the corresponding samples from the embedding
        sampled_embedding = embedding[sampled_indices]

        return sampled_embedding

    def get_sample_indices(self):
        """
        Return the indices of the sampled data points in the original dataset.
        """
        if self.sample_indices is None:
            raise ValueError("No samples have been selected. Please run stratified_sampling first.")
        return self.sample_indices

    def plot(self, embedding):
        if self.n_components == 2:
            plt.scatter(embedding[:, 0], embedding[:, 1], c='blue', marker='o')
            plt.title('UMAP Projection (2D)')
            plt.xlabel('UMAP 1')
            plt.ylabel('UMAP 2')
            plt.show()

        elif self.n_components == 3:
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
            ax.scatter(embedding[:, 0], embedding[:, 1], embedding[:, 2], c='blue', marker='o')
            ax.set_title('UMAP Projection (3D)')
            ax.set_xlabel('UMAP 1')
            ax.set_ylabel('UMAP 2')
            ax.set_zlabel('UMAP 3')
            plt.show()



##### HDBSCAN - CLUSTERING BASED SAMPLING

In [4]:
class ClusterSampler:
    def __init__(self, data, sampling_percent=10, **hdbscan_params):
        """
        Initialize the ClusterSampler class.

        Parameters:
        - data: DataFrame containing the dataset
        - sampling_percent: Percentage of points to sample from each cluster (0-100)
        - **hdbscan_params: Additional parameters to pass to HDBSCAN
        """
        self.data = self._convert_data_types(data)
        self.sampling_percent = sampling_percent
        self.is_categorical = self._detect_categorical(data)
        self.cluster_labels = None
        self.hdbscan_params = hdbscan_params

    def _convert_data_types(self, data):
        """Ensure continuous columns are float64 and categorical columns are object."""
        continuous_cols = data.select_dtypes(include=['float', 'int']).columns
        data[continuous_cols] = data[continuous_cols].astype(np.float64)

        categorical_cols = data.select_dtypes(include=['object', 'category']).columns
        data[categorical_cols] = data[categorical_cols].astype('object')

        return data

    def _detect_categorical(self, data):
        """Detect if the dataset contains categorical features."""
        return data.select_dtypes(include=['object', 'category']).shape[1] > 0

    def _compute_distance_matrix(self):
        """Compute the distance matrix based on the data type."""
        if self.is_categorical:
            gower_matrix = gower.gower_matrix(self.data)
            return gower_matrix.astype(np.float64)
        else:
            return pairwise_distances(self.data, metric='euclidean')

    def fit_clusters(self):
        """Fit HDBSCAN on the dataset with appropriate distance metric."""
        distance_matrix = self._compute_distance_matrix()
        clusterer = hdbscan.HDBSCAN(metric='precomputed' if self.is_categorical else 'euclidean', **self.hdbscan_params)
        self.cluster_labels = clusterer.fit_predict(distance_matrix)

    def sample_points(self):
        """Sample a representative subset of points from each cluster, including noise points as a separate cluster."""
        if self.cluster_labels is None:
            raise ValueError("Clusters have not been computed. Call fit_clusters() first.")

        data_with_labels = self.data.copy()
        data_with_labels['cluster'] = self.cluster_labels

        sampled_indices = []
        unique_labels = np.unique(self.cluster_labels)

        for cluster_label in unique_labels:
            cluster_indices = data_with_labels[data_with_labels['cluster'] == cluster_label].index
            sample_size = max(1, int(len(cluster_indices) * (self.sampling_percent / 100)))

            # Avoid sampling more points than available
            if len(cluster_indices) < sample_size:
                print(f"Cluster {cluster_label} has only {len(cluster_indices)} points, sampling {len(cluster_indices)}.")
            sampled_indices.extend(np.random.choice(cluster_indices, min(sample_size, len(cluster_indices)), replace=False))

        # Ensure sampled indices are unique and valid
        sampled_indices = list(set(sampled_indices))
        sampled_indices = [idx for idx in sampled_indices if idx < len(self.data)]

        return sampled_indices

    def run(self):
        """Execute the full sampling pipeline: cluster, then sample."""
        self.fit_clusters()
        return self.sample_points()


##### ISOLATION FOREST AND KS STATISTIC SAMPLER

In [5]:
class AnomalySampler:
    def __init__(self, X, sample_weight=None):
        """
        Initialize the AnomalySampler class.

        Parameters:
        - X: DataFrame containing the dataset
        - sample_weight: Optional sample weights for the Isolation Forest
        """
        self.original_data = X
        self.X = self._one_hot_encode(X)
        self.sample_weight = sample_weight
        self.preds = self.isolation_forest(self.X, sample_weight)

    def _one_hot_encode(self, data):
        """
        Apply one-hot encoding to categorical columns.

        Parameters:
        - data: DataFrame containing the dataset

        Returns:
        - One-hot encoded DataFrame
        """
        return pd.get_dummies(data, drop_first=True)  # drop_first to avoid multicollinearity, if relevant

    @staticmethod
    def isolation_forest(X, sample_weight=None):
        """
        Fits an Isolation Forest to the dataset and assigns an anomaly score to each sample.

        Parameters:
        - X: DataFrame or array-like containing the dataset
        - sample_weight: Optional sample weights for the Isolation Forest

        Returns:
        - preds: Anomaly scores for each sample
        """
        clf = IsolationForest().fit(X, sample_weight=sample_weight)
        preds = clf.score_samples(X)
        return preds

    @staticmethod
    def get_5_percent(num):
        """Calculate 5% of a given number."""
        return round(5 / 100 * num)

    def get_5_percent_splits(self, length):
        """Splits a given length into 5% intervals."""
        five_percent = self.get_5_percent(length)
        return np.arange(five_percent, length, five_percent)

    def find_sample_indices(self):
        """
        Finds a sample by comparing the distribution of anomaly scores between the sample
        and the original distribution using the KS-test. Starts with a 5% sample, increasing
        by 5% increments until a significant sample (p-value > 0.95) is found or a limit is reached.

        Returns:
        - List of indices representing the sample in the original dataset
        """
        size_splits = self.get_5_percent_splits(len(self.X))
        element = 1
        iteration = 0

        while element < len(size_splits):
            sample_size = size_splits[element]
            sample_indices = np.random.choice(np.arange(self.preds.size), size=sample_size, replace=False)
            sample = np.take(self.preds, sample_indices)

            # Check if KS test p-value indicates similar distributions
            if ks_2samp(self.preds, sample).pvalue > 0.95:
                return sample_indices  # Return indices from the original dataset

            iteration += 1
            if iteration >= 20:
                element += 1
                iteration = 0

        # If no suitable sample is found, return the last attempted sample indices
        return sample_indices



#### ENTROPY SAMPLER

In [6]:
class EntropySampler:
    def __init__(self, data, sampling_percent=10, bins=10):
        """
        Initialize the EntropySampler class.

        Parameters:
        - data: DataFrame containing the dataset
        - sampling_percent: Percentage of points to sample based on entropy (0-100)
        - bins: Number of bins to use for continuous data entropy calculation
        """
        self.data = data
        self.sampling_percent = sampling_percent
        self.bins = bins
        self.entropy_scores = None

    def _calculate_entropy(self):
        """
        Calculate the entropy for each feature and aggregate entropy per data point.

        Returns:
        - entropy_scores: Array of entropy scores for each data point
        """
        entropy_scores = np.zeros(len(self.data))

        for col in self.data.columns:
            if pd.api.types.is_numeric_dtype(self.data[col]):
                # For continuous data, bin it and calculate entropy over the bins
                counts, _ = np.histogram(self.data[col], bins=self.bins)
                feature_entropy = entropy(counts + 1e-10)  # Add small value to avoid log(0)
                feature_contributions = np.digitize(self.data[col], bins=np.histogram_bin_edges(self.data[col], bins=self.bins))
            else:
                # For categorical data, calculate entropy over unique values
                counts = self.data[col].value_counts().values
                feature_entropy = entropy(counts + 1e-10)
                feature_contributions = self.data[col].map(self.data[col].value_counts(normalize=True)).values

            # Accumulate entropy scores based on the feature contributions for each data point
            entropy_scores += feature_contributions * feature_entropy

        self.entropy_scores = entropy_scores

    def sample_indices(self):
        """
        Get indices of points with the highest entropy scores.

        Returns:
        - List of indices for sampled points based on entropy scores
        """
        if self.entropy_scores is None:
            self._calculate_entropy()

        # Determine the number of points to sample
        num_samples = max(1, int(len(self.data) * (self.sampling_percent / 100)))

        # Get the indices of the top entropy scores
        top_indices = np.argsort(self.entropy_scores)[-num_samples:]

        return top_indices

    def run(self):
        """
        Execute the full entropy-based sampling process.

        Returns:
        - List of indices for sampled points based on entropy scores
        """
        self._calculate_entropy()
        return self.sample_indices()

### DISTANCE BASED SAMPLER

In [7]:
class DistanceBasedSampler:
    def __init__(self, data, k=5, sampling_percent=10):
        self.data = data.reset_index(drop=True)  # Reset index for consistent access
        self.k = k
        self.sampling_percent = sampling_percent
        self.cluster_labels = None
        self.cluster_centroids = None

    def _prepare_data(self):
        """Ensure all categorical columns are treated as strings."""
        self.data = self.data.apply(lambda col: col.astype(str) if col.dtype == 'object' else col)
        return self.data

    def fit_clusters(self):
        """
        Fit clusters using K-Prototypes if categorical data is present,
        otherwise fallback to K-Means for purely numerical data.
        """
        self._prepare_data()
        data_array = self.data.values

        # Identify categorical column indices
        categorical_indices = [
            i for i, col in enumerate(self.data.columns) if self.data.dtypes[col] == 'object'
        ]

        if len(categorical_indices) == 0:
            # No categorical data, use K-Means
            print("No categorical data detected. Using K-Means instead.")
            kmeans = KMeans(n_clusters=self.k, random_state=42)
            clusters = kmeans.fit_predict(data_array)
            self.cluster_labels = clusters
            self.cluster_centroids = kmeans.cluster_centers_
        else:
            # Use K-Prototypes for mixed data
            kproto = KPrototypes(n_clusters=self.k, init='Cao', random_state=42)
            clusters = kproto.fit_predict(data_array, categorical=categorical_indices)
            self.cluster_labels = clusters
            self.cluster_centroids = kproto.cluster_centroids_

        return clusters

    def _calculate_mixed_distance(self, point, centroids):
        """
        Calculate the mixed distance (numerical + categorical) from a point to centroids.
        """
        numeric_cols = self.data.select_dtypes(include=[np.number]).columns
        categorical_cols = self.data.select_dtypes(include=['object']).columns

        point_numeric = point[numeric_cols].values.astype(float)
        point_categorical = point[categorical_cols].values

        distances = []
        for centroid in centroids:
            # Split centroids into numeric and categorical parts
            centroid_numeric = np.array(centroid[:len(numeric_cols)], dtype=float)
            centroid_categorical = np.array(centroid[len(numeric_cols):])

            # Calculate numerical distance (Euclidean)
            numeric_distance = np.linalg.norm(point_numeric - centroid_numeric)
            # Calculate categorical distance (Hamming)
            categorical_distance = np.sum(point_categorical != centroid_categorical)

            distances.append(numeric_distance + categorical_distance)

        return np.array(distances)

    def sample_indices(self):
        """
        Identify sample indices based on distance to centroids.
        """
        if self.cluster_labels is None:
            self.fit_clusters()

        centroids = self.cluster_centroids
        distances = np.zeros(len(self.data))

        for i, (_, point) in enumerate(self.data.iterrows()):
            distances[i] = np.min(self._calculate_mixed_distance(point, centroids))

        num_samples = max(1, int(len(self.data) * (self.sampling_percent / 100)))
        sampled_indices = np.argsort(distances)[-num_samples:]

        return sampled_indices

    def run(self):
        """
        Run the sampling process.
        """
        self.fit_clusters()
        return self.sample_indices()

#### RANDOM SAMPLING

In [8]:
class RandomSampler:
    def __init__(self, data, sampling_percent=10):
        """
        Initialize the RandomSampler class.

        Parameters:
        - data: DataFrame containing the dataset
        - sampling_percent: Percentage of rows to sample (0-100)
        """
        self.data = data.reset_index(drop=True)  # Reset index to ensure consistency
        self.sampling_percent = sampling_percent

    def sample_indices(self):
        """
        Randomly sample indices of rows from the dataset.

        Returns:
        - A list of sampled indices.
        """
        if not (0 <= self.sampling_percent <= 100):
            raise ValueError("sampling_percent must be between 0 and 100.")

        # Calculate the number of samples to draw
        sample_size = int(len(self.data) * (self.sampling_percent / 100))
        sample_size = max(1, sample_size)  # Ensure at least one index is selected

        # Perform random sampling without replacement and return indices
        sampled_indices = self.data.sample(n=sample_size, random_state=42).index.tolist()
        return sampled_indices

### MACHINE LEARNING MODELS

In [9]:
# !mkdir Data

#### READING DATA

In [10]:
# Custom transformer to drop highly correlated features
class DropHighCorrelation(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.6):
        """
        Custom transformer to drop highly correlated features.

        Parameters:
        - threshold: Correlation threshold above which features will be dropped.
        """
        self.threshold = threshold
        self.to_drop = None

    def fit(self, X, y=None):
        """
        Identify features to drop based on the correlation threshold.
        """
        corr_matrix = pd.DataFrame(X).corr().abs()  # Compute the absolute correlation matrix
        upper_tri = corr_matrix.where(
            np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
        )  # Extract upper triangle
        self.to_drop = [
            column for column in upper_tri.columns if any(upper_tri[column] > self.threshold)
        ]

        print(f"DropHighCorrelation: {len(self.to_drop)} columns will be dropped due to correlation (threshold={self.threshold}).")
        return self

    def transform(self, X):
        """
        Drop the identified features from the dataset.
        """
        return pd.DataFrame(X).drop(columns=self.to_drop, errors='ignore')

# Main ModelTrainer class
class ModelTrainer:
    def __init__(self, datasets_X, datasets_y, task_type, dataset_names=None):
        """
        Initializes the ModelTrainer with datasets, explicitly provided task type, and dataset names.

        Parameters:
        - datasets_X: List of [X_train, X_test] for different datasets
        - datasets_y: List of [y_train, y_test] for different datasets
        - task_type: A string explicitly specifying the task type, either 'regression' or 'classification'.
        - dataset_names: List of names for the datasets to be used as index in the results DataFrame
        """
        if task_type not in ['classification', 'regression']:
            raise ValueError("task_type must be either 'classification' or 'regression'")

        self.datasets_X = datasets_X
        self.datasets_y = datasets_y
        self.task_type = task_type
        self.dataset_names = dataset_names

    def _select_model(self):
        """Select model based on task type."""
        if self.task_type == 'classification':
            return {
                'RandomForest': RandomForestClassifier(random_state=42),
                'GradientBoosting': GradientBoostingClassifier(random_state=42),
                'LogisticRegression': LogisticRegression(),
                'XGBoost': xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
            }
        elif self.task_type == 'regression':
            return {
                'RandomForest': RandomForestRegressor(random_state=42),
                'GradientBoosting': GradientBoostingRegressor(random_state=42),
                'LinearRegression': LinearRegression(),
                'XGBoost': xgb.XGBRegressor(random_state=42)
            }

    def _create_pipeline(self, model, X):
        """Create a preprocessing and modeling pipeline."""
        # Identify categorical and numerical features
        categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()
        numerical_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

        # Preprocessing for numerical features: Standard scaling
        numerical_transformer = StandardScaler()

        # Preprocessing for categorical features: One-hot encoding
        categorical_transformer = OneHotEncoder(handle_unknown='ignore', drop='first', sparse_output=False)

        # Combine preprocessors in a column transformer
        preprocessor = ColumnTransformer(
            transformers=[
                ('num', numerical_transformer, numerical_features),
                ('cat', categorical_transformer, categorical_features)
            ]
        )

        # Define a pipeline with preprocessing, correlation dropping, and the specified model
        pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),  # First preprocess
            ('drop_high_corr', DropHighCorrelation(threshold=0.6)),  # Then drop highly correlated features
            ('model', model)  # Finally, apply the model
        ])
        return pipeline

    def _get_best_cutoff(self, y_true, y_pred_proba):
        """Use Youden's J statistic to determine the best cutoff point for classification."""
        fpr, tpr, thresholds = roc_curve(y_true, y_pred_proba)
        youden_index = tpr - fpr
        best_cutoff = thresholds[np.argmax(youden_index)]
        return best_cutoff

    def _train_and_evaluate(self, model, X_train, X_test, y_train, y_test):
        """Train the model and evaluate it on both the training and test datasets."""
        start_time = time.time()
        model.fit(X_train, y_train)
        training_time = time.time() - start_time

        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)

        if self.task_type == 'classification':
            y_pred_proba_test = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else y_pred_test
            y_pred_proba_train = model.predict_proba(X_train)[:, 1] if hasattr(model, 'predict_proba') else y_pred_train

            best_cutoff = self._get_best_cutoff(y_test, y_pred_proba_test)
            y_pred_class_test = (y_pred_proba_test >= best_cutoff).astype(int)
            y_pred_class_train = (y_pred_proba_train >= best_cutoff).astype(int)

            metrics = {
                'Train F1': f1_score(y_train, y_pred_class_train),
                'Test F1': f1_score(y_test, y_pred_class_test),
                'Train AUC': roc_auc_score(y_train, y_pred_proba_train),
                'Test AUC': roc_auc_score(y_test, y_pred_proba_test),
                'Train Accuracy': accuracy_score(y_train, y_pred_class_train),
                'Test Accuracy': accuracy_score(y_test, y_pred_class_test),
                'Train Recall': recall_score(y_train, y_pred_class_train),
                'Test Recall': recall_score(y_test, y_pred_class_test),
                'Train Precision': precision_score(y_train, y_pred_class_train),
                'Test Precision': precision_score(y_test, y_pred_class_test),
                'Best Cutoff': best_cutoff,
                'Training Time (seconds)': training_time
            }
        else:
            metrics = {
                'Train MSE': mean_squared_error(y_train, y_pred_train),
                'Test MSE': mean_squared_error(y_test, y_pred_test),
                'Train MAPE': mean_absolute_percentage_error(y_train, y_pred_train),
                'Test MAPE': mean_absolute_percentage_error(y_test, y_pred_test),
                'Train R2': r2_score(y_train, y_pred_train),
                'Test R2': r2_score(y_test, y_pred_test),
                'Training Time (seconds)': training_time
            }

        return metrics

    def train_models(self):
        """Train and evaluate models on multiple datasets and return a DataFrame of results."""
        models = self._select_model()
        results = []

        for idx, (X_data, y_data) in enumerate(zip(self.datasets_X, self.datasets_y)):
            X_train, X_test = X_data
            y_train, y_test = y_data

            for model_name, model in models.items():
                pipeline = self._create_pipeline(model, X_train)
                metrics = self._train_and_evaluate(pipeline, X_train, X_test, y_train, y_test)
                metrics['Dataset'] = self.dataset_names[idx] if self.dataset_names else f'Dataset {idx+1}'
                metrics['Model'] = model_name
                results.append(metrics)

        return pd.DataFrame(results).set_index('Dataset')

In [11]:
train = pd.read_csv('/content/Data/TUANDROMD.csv')

# Display the first few rows of the training data
print("Training Data:")

train = train[~(train['Label'].isnull())]
train.columns = train.columns.str.replace(r'[^a-zA-Z0-9_]', '_', regex=True)

X = train.drop(columns=['Label'])
y = train['Label']

categorical_features = X.select_dtypes(include=['object']).columns.tolist()
# Convert categorical features to category dtype
for feature in categorical_features:
    X[feature] = X[feature].astype('category')

X.head()


Training Data:


,ACCESS_ALL_DOWNLOADS,ACCESS_CACHE_FILESYSTEM,ACCESS_CHECKIN_PROPERTIES,ACCESS_COARSE_LOCATION,ACCESS_COARSE_UPDATES,ACCESS_FINE_LOCATION,ACCESS_LOCATION_EXTRA_COMMANDS,ACCESS_MOCK_LOCATION,ACCESS_MTK_MMHW,ACCESS_NETWORK_STATE,...,Landroid_content_pm_PackageManager___getInstalledPackages,Landroid_telephony_TelephonyManager___getLine1Number,Landroid_telephony_TelephonyManager___getNetworkOperator,Landroid_telephony_TelephonyManager___getNetworkOperatorName,Landroid_telephony_TelephonyManager___getNetworkCountryIso,Landroid_telephony_TelephonyManager___getSimOperator,Landroid_telephony_TelephonyManager___getSimOperatorName,Landroid_telephony_TelephonyManager___getSimCountryIso,Landroid_telephony_TelephonyManager___getSimSerialNumber,Lorg_apache_http_impl_client_DefaultHttpClient___execute
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

In [13]:
#### SAMPLING METHODS ####
data = X_train1
# ================================ #
##### DIM REDUCTION AND GRID SEARCH
# ================================ #
start_time = time.time()
stratified_sampler = StratifiedSamplingUMAP(n_neighbors=15, min_dist=0.1, n_components=3, n_grids=10, sample_percentage=0.3)
embedding = stratified_sampler.fit_transform(data)
sampled_embedding = stratified_sampler.stratified_sampling(embedding)
sample_indices = stratified_sampler.get_sample_indices()
print("Sampled Data:", len(sample_indices),"Data:",len(data))
X_train1_1 = data.iloc[sample_indices]
y_train1_1 = y_train1.iloc[sample_indices]
print("Time taken for Dim Red and Grid Search Sampling:", time.time() - start_time, "seconds")

# # ================================ #
# ##### HDBSCAN - CLUSTERING BASED SAMPLING
# # ================================ #

# start_time = time.time()
# sampler = ClusterSampler(data, sampling_percent=30, min_cluster_size=15, min_samples=3)
# sampled_indices = sampler.run()
# X_train1_2 = data.loc[sampled_indices]
# y_train1_2 = y_train1.iloc[sampled_indices]
# print("Sampled Data:", len(sampled_indices),"Data:",len(data))
# print("Time taken for Clustering Based Sampling:", time.time() - start_time, "seconds")

# ================================ #
##### ISOLATION FOREST AND KS STATISTIC SAMPLER #####
# ================================ #

start_time = time.time()
sampler = AnomalySampler(data)
sample_indices = sampler.find_sample_indices()
X_train1_3 = data.iloc[sample_indices]
y_train1_3 = y_train1.iloc[sample_indices]
print("Sampled Data:", len(sample_indices),"Data:",len(data))
print("Time taken for Isolation Forest and KS Sampling:", time.time() - start_time, "seconds")

# ================================ #
##### ENTROPY SAMPLER #####
# ================================ #

start_time = time.time()
sampler = EntropySampler(data,sampling_percent=30)
sample_indices = sampler.run()
X_train1_4 = data.iloc[sample_indices]
y_train1_4 = y_train1.iloc[sample_indices]
print("Sampled Data:", len(sample_indices),"Data:",len(data))
print("Time taken for Entropy Sampling:", time.time() - start_time, "seconds")

# ================================ #
##### DISTANCE BASED SAMPLER #####
# ================================ #

start_time = time.time()
sampler = DistanceBasedSampler(data, k=5, sampling_percent=30)
sampled_indices = sampler.run()
X_train1_5 = data.iloc[sampled_indices]
y_train1_5 = y_train1.iloc[sampled_indices]
print("Sampled Data:", len(sampled_indices),"Data:",len(data))
print("Time taken for Distance Based Sampling:", time.time() - start_time, "seconds")

# ================================ #
##### RANDOM SAMPLER #####
# ================================ #

start_time = time.time()
sampler = RandomSampler(data=data, sampling_percent=30)
sampled_indices = sampler.sample_indices()
X_train1_6 = X_train1.iloc[sampled_indices]
y_train1_6 = y_train1.iloc[sampled_indices]
print("Sampled Data:", len(sampled_indices),"Data:",len(data))
print("Time taken for Random Sampling:", time.time() - start_time, "seconds")

No categorical variables detected. Using 'euclidean' metric for UMAP.
Sampled Data: 1071 Data: 3571
Time taken for Dim Red and Grid Search Sampling: 16.923755407333374 seconds
Sampled Data: 358 Data: 3571
Time taken for Isolation Forest and KS Sampling: 0.1495518684387207 seconds
Sampled Data: 1071 Data: 3571
Time taken for Entropy Sampling: 0.15165162086486816 seconds
No categorical data detected. Using K-Means instead.
Sampled Data: 1071 Data: 3571
Time taken for Distance Based Sampling: 3.027362108230591 seconds
Sampled Data: 1071 Data: 3571
Time taken for Random Sampling: 0.004820108413696289 seconds


In [14]:
# ================================ #
##### Trainer for All State Sampling Methods
# ================================ #
trainer = ModelTrainer(
    datasets_X=[
        [X_train1_1, X_test1],
        # [X_train1_2, X_test1],
        [X_train1_3, X_test1],
        [X_train1_4, X_test1],
        [X_train1_5, X_test1],
        [X_train1_6, X_test1]
    ],
    datasets_y=[
        [y_train1_1, y_test1],
        # [y_train1_2, y_test1],
        [y_train1_3, y_test1],
        [y_train1_4, y_test1],
        [y_train1_5, y_test1],
        [y_train1_6, y_test1]
    ],
    task_type='regression',
    dataset_names=[
        'Dim Reduction',
        'Clustering',
        'Anomaly Detection',
        'Entropy',
        'Distance Based',
        'Random'
    ]
)

results = trainer.train_models()

DropHighCorrelation: 153 columns will be dropped due to correlation (threshold=0.6).
DropHighCorrelation: 153 columns will be dropped due to correlation (threshold=0.6).
DropHighCorrelation: 153 columns will be dropped due to correlation (threshold=0.6).
DropHighCorrelation: 153 columns will be dropped due to correlation (threshold=0.6).
DropHighCorrelation: 150 columns will be dropped due to correlation (threshold=0.6).
DropHighCorrelation: 150 columns will be dropped due to correlation (threshold=0.6).
DropHighCorrelation: 150 columns will be dropped due to correlation (threshold=0.6).
DropHighCorrelation: 150 columns will be dropped due to correlation (threshold=0.6).
DropHighCorrelation: 151 columns will be dropped due to correlation (threshold=0.6).
DropHighCorrelation: 151 columns will be dropped due to correlation (threshold=0.6).
DropHighCorrelation: 151 columns will be dropped due to correlation (threshold=0.6).
DropHighCorrelation: 151 columns will be dropped due to correlati

In [15]:
results.to_csv("TUANDROMD.csv")

In [16]:
results

,Train MSE,Test MSE,Train MAPE,Test MAPE,Train R2,Test R2,Training Time (seconds),Model
Dataset,,,,,,,,
Dim Reduction,0.011043,1.696481e-02,6.945666e+13,1.012350e+14,0.933236,8.945880e-01,0.270201,RandomForest
Dim Reduction,0.019962,2.181607e-02,1.216205e+14,1.354945e+14,0.879317,8.644444e-01,0.227483,GradientBoosting
Dim Reduction,0.033152,5.130240e+20,2.159873e+14,3.413564e+24,0.799571,-3.187708e+21,0.139143,LinearRegression
Dim Reduction,0.009204,1.666194e-02,4.969542e+13,1.038753e+14,0.944358,8.964699e-01,0.304795,XGBoost
Clustering,0.010923,2.782190e-02,8.051510e+13,1.721228e+14,0.925596,8.271268e-01,0.152040,RandomForest
Clustering,0.011085,2.770502e-02,7.976831e+13,1.924711e+14,0.924493,8.278531e-01,0.121192,GradientBoosting
Clustering,0.025015,4.187184e-02,1.609937e+14,2.733408e+14,0.829615,7.398266e-01,0.066872,LinearRegression
Clustering,0.007112,3.217870e-02,3.259169e+13,1.801986e+14,0.951556,8.000555e-01,0.211442,XGBoost
Anomaly Detection,0.001228,1.028265e-01,2.207647e+13,9.067718e+13,0.994147,3.610806e-01,0.273703,RandomForest
